# Cleaning-2-feature engineering

In [12]:
#importamos las librerias 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.datasets import load_digits

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import root_mean_squared_error

 
from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [44]:
paht= './data/diamonds_train.csv'   #leemos el archivos csv que esta en la carpeta data
diamons_cleaning_2 = pd.read_csv(paht, sep= ",")

In [45]:
#Volumen: Calcular el volumen del diamante como una nueva característica:
diamons_cleaning_2['volume'] = diamons_cleaning_2['x'] * diamons_cleaning_2['y'] * diamons_cleaning_2['z']


In [46]:
#Volvemos las variables categoricas en numericas 
diamons_cleaning_2 = pd.get_dummies(diamons_cleaning_2, columns=["city","cut","color","clarity",], dtype=int)

In [47]:
diamons_cleaning_2

,price,carat,depth,table,x,y,z,volume,city_Amsterdam,city_Antwerp,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,4268,1.21,62.4,58.0,6.83,6.79,4.25,197.096725,0,0,...,0,1,0,0,0,0,0,1,0,0
1,505,0.32,63.0,57.0,4.35,4.38,2.75,52.395750,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2686,0.71,65.5,55.0,5.62,5.53,3.65,113.436890,0,0,...,0,0,0,0,0,0,1,0,0,0
3,738,0.41,63.8,56.0,4.68,4.72,3.00,66.268800,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4882,1.02,60.5,59.0,6.55,6.51,3.95,168.429975,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,62.7,57.0,7.10,7.04,4.43,221.429120,0,1,...,0,0,0,0,0,0,1,0,0,0
40451,12615,2.02,57.1,60.0,8.31,8.25,4.73,324.276975,0,0,...,0,0,0,0,0,1,0,0,0,0
40452,5457,1.01,62.7,56.0,6.37,6.42,4.01,163.990554,0,0,...,0,0,0,0,1,0,0,0,0,0
40453,456,0.33,61.9,54.3,4.45,4.47,2.76,54.900540,0,0,...,0,1,0,0,0,0,1,0,0,0


# Probando modelos cleaning-2 (Los mejores de la primera prueba)
### Dividimos datos de entranmiento

In [48]:
#Separar las características (X) y el objetivo (y):
X= diamons_cleaning_2.drop('price', axis=1)
y = diamons_cleaning_2['price']

#Dividir el conjunto de datos en entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [49]:
#LightGBM:
model_5_cleaning_2 = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=43)



In [50]:
model_5_cleaning_2.fit(X, y)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.089043 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1581
[LightGBM] [Info] Number of data points in the train set: 40455, number of used features: 40
[LightGBM] [Info] Start training from score 3928.444469


LGBMRegressor(random_state=43)

In [52]:
y_pred_5_cleaning_2 = model_5_cleaning_2.predict(X_test)
rmse_2 = np.sqrt(mean_squared_error(y_test, y_pred_5_cleaning_2))
rmse_2

np.float64(480.25531699836546)

In [51]:
%%time
# Realizar validación cruzada y calcular el RMSE
scores = cross_val_score(model_5_cleaning_2, 
                         X, 
                         y, 
                         scoring='neg_root_mean_squared_error', 
                         cv=5,
                         n_jobs=-1)

# Imprimir el tipo de modelo y los scores
print(type(model_5_cleaning_2), '\n')
scores, 

# Calcular y mostrar el RMSE promedio
print(np.mean(-scores))


<class 'lightgbm.sklearn.LGBMRegressor'> 

549.6441891963627
CPU times: total: 125 ms
Wall time: 7.03 s


# Datos del test de kaggel
### Hacemos el mismos feature engineering

In [54]:
paht= './data/diamonds_test.csv'
diamons_test = pd.read_csv(paht, sep= ",")

In [55]:
#Volumen: Calcular el volumen del diamante como una nueva característica:
diamons_test['volume'] = diamons_test['x'] * diamons_test['y'] * diamons_test['z']


In [56]:
#Volvemos las variables categoricas en numericas 
diamons_test = pd.get_dummies(diamons_test, columns=["city","cut","color","clarity",], dtype=int)

In [57]:
#dropeamos el indice
diamons_test = diamons_test.drop("id", axis=1)

In [58]:
diamons_test

,carat,depth,table,x,y,z,volume,city_Amsterdam,city_Antwerp,city_Dubai,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.79,62.7,60.0,5.82,5.89,3.67,125.806866,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1.20,61.0,57.0,6.81,6.89,4.18,196.129362,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,1.57,62.2,61.0,7.38,7.32,4.57,246.878712,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0.90,63.8,54.0,6.09,6.13,3.90,145.593630,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0.50,62.9,58.0,5.05,5.09,3.19,81.997355,1,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,61.9,56.0,5.35,5.32,3.30,93.924600,1,0,0,...,0,0,0,0,1,0,0,0,0,0
13481,0.71,62.2,55.0,5.71,5.73,3.56,116.477148,0,0,0,...,1,0,0,0,0,0,0,1,0,0
13482,0.70,61.6,55.0,5.75,5.71,3.53,115.898725,0,0,0,...,0,0,0,0,0,0,1,0,0,0
13483,0.70,58.8,57.0,5.85,5.89,3.45,118.874925,0,0,0,...,0,0,0,0,0,1,0,0,0,0


# Pasamos los datos de prueba por el modelo

In [60]:
#paso 3
y_pred_test = model_5_cleaning_2.predict(diamons_test)
y_pred_test

array([2854.94197463, 5740.11785321, 9566.16606086, ..., 3124.72214685,
       2216.04283461,  812.80677082])

In [62]:
# Convertir el array de NumPy en un DataFrame de pandas
y_pred_test_df = pd.DataFrame(y_pred_test, columns=['Price'])
y_pred_test_df['id'] = y_pred_test_df.index
# Mostrar las primeras filas del DataFrame
y_pred_test_df.head()


,Price,id
0,2854.941975,0
1,5740.117853,1
2,9566.166061,2
3,3956.349394,3
4,1617.197673,4


In [63]:
#paso 5

# Guardar el DataFrame en un archivo CSV en la carpeta 'predictions'
y_pred_test_df.to_csv('predictions/cleaning-2-model-LightGBM:.csv', index=False)

print("El DataFrame ha sido guardado en la carpeta 'predictions'.")



El DataFrame ha sido guardado en la carpeta 'predictions'.
